In [1]:
from concatmatching import *
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
# Distance-5 color code check matrix under bit-flip noise
H = np.array([[0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]])

In [3]:
decoder = Decoder(H, 
                  p=None,
                  comparison=True,
#                   check_colors=[2, 2, 0, 0, 1, 1, 2, 0, 1],
                  verbose=True)

9 checks, 19 faults
p given: False
filtering_strategy = greedy_coloring
filtering_options = {}
comparison = True
Start decomposition.

ROUND 0:
CHILD DECOMP 0
6 checks, 9 edges, max degree = 2
CHILD DECOMP 1
6 checks, 9 edges, max degree = 2
CHILD DECOMP 2
6 checks, 9 edges, max degree = 2

ROUND 1 (DECOMP 0):
12 checks, 19 edges

ROUND 1 (DECOMP 1):
12 checks, 19 edges

ROUND 1 (DECOMP 2):
12 checks, 19 edges


**Note on the `check_colors` parameter:**

By default, the `greedy_coloring` strategy is used for decomposing the tanner graph into matchable tanner graphs. It works by assigning colours to checks in a way that each fault affects checks of distinct colours, using a graph colouring subroutine. Then each pair of colours can be used to extract a matchable tanner graph from the original tanner graph. 

However, this is not always desirable. For example, when decoding a 2D color code under bit-flip noise (as in the above example), this method works well. However, if it has measurement errors on syndrome qubits, each of them affects only two checks (in consecutive time slices), and assigning the same colour to them doesn't matter for extracting a matchable tanner graph. In such cases, you can manually give the colours of checks by parameter `check_colors`. (Currently support only three colours, so it should be composed of 0, 1, and 2 only.)

In [7]:
# Generating a random syndrome data
errors = np.random.choice([True, False], p=(0.2, 0.8), size=H.shape[1])
syndrome = errors@H.T
print(errors.sum(), syndrome.sum())

4 8


In [8]:
# Decoding
preds, data = decoder.decode(syndrome, verbose=True, full_output=True, check_validity=True)

>> Decoding DECOMP 0...
num_stages = 2
Start decoding.
Stage 0... Success!
Stage 1... Success!
Valid: True
Weight = 3.0

>> Decoding DECOMP 1...
num_stages = 2
Start decoding.
Stage 0... Success!
Stage 1... Success!
Valid: True
Weight = 5.0

>> Decoding DECOMP 2...
num_stages = 2
Start decoding.
Stage 0... Success!
Stage 1... Success!
Valid: True
Weight = 3.0

Best DECOMP: 0
Min weight = 3.0


In [9]:
preds

array([False,  True, False, False, False, False, False, False,  True,
       False, False, False, False,  True, False, False, False, False,
       False])

In [10]:
# Decoder can be saved & loaded
decoder.save('decoder')
decoder = Decoder.load('decoder')